# Example Experiment
> Experiment using Repeated MNIST and BatchBALD vs BALD vs random sampling

This notebook ties everything together and runs an AL loop.

In [ ]:
# experiment

import blackhc.project.script
from tqdm.auto import tqdm

In [ ]:
# experiment

import torch
import math
from torch import nn as nn
from torch.nn import functional as F

from batchbald_redux import active_learning, batchbald, consistent_mc_dropout, joint_entropy, repeated_mnist

Let's define our Bayesian CNN model that we will use to train MNIST.

In [ ]:
# experiment

class BayesianCNN(consistent_mc_dropout.BayesianModule):
    def __init__(self, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv1_drop = consistent_mc_dropout.ConsistentMCDropout2d()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = consistent_mc_dropout.ConsistentMCDropout2d()
        self.fc1 = nn.Linear(1024, 128)
        self.fc1_drop = consistent_mc_dropout.ConsistentMCDropout()
        self.fc2 = nn.Linear(128, num_classes)

    def mc_forward_impl(self, input: torch.Tensor):
        input = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(input)), 2))
        input = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(input)), 2))
        input = input.view(-1, 1024)
        input = F.relu(self.fc1_drop(self.fc1(input)))
        input = self.fc2(input)
        input = F.log_softmax(input, dim=1)

        return input

Grab our dataset, we'll use Repeated-MNIST. We will acquire to samples for each class for our initial training set.

In [ ]:
# experiment

train_dataset, test_dataset = repeated_mnist.create_repeated_MNIST_dataset(num_repetitions=1, add_noise=False)

num_initial_samples = 20
num_classes = 10

initial_samples = active_learning.get_balanced_sample_indices(
    repeated_mnist.get_targets(train_dataset),
    num_classes=num_classes,
    n_per_digit=num_initial_samples/num_classes)

For this example, we are going to take two shortcuts that will reduce the performance:
* we discard most of the training set and only keep 20k samples; and
* we don't implement early stopping or epoch-wise training.

Instead, we always train by drawing 24576 many samples from the training set. This will overfit in the beginning and underfit later, but it still is sufficient to achieve 90% accuracy with 105 samples in the training set. 

In [ ]:
# experiment
max_training_samples = 150
acquisition_batch_size = 5
num_inference_samples = 100
num_test_inference_samples = 5
num_samples = 100000

test_batch_size = 512
batch_size = 64
scoring_batch_size = 128
training_iterations = 4096 * 6

use_cuda = torch.cuda.is_available()

print(f"use_cuda: {use_cuda}")

device = "cuda" if use_cuda else "cpu"

kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=test_batch_size,
                                          shuffle=False,
                                          **kwargs)

active_learning_data = active_learning.ActiveLearningData(train_dataset)

# Split off the initial samples first.
active_learning_data.acquire(initial_samples)

# THIS REMOVES MOST OF THE POOL DATA. UNCOMMENT THIS TO TAKE ALL UNLABELLED DATA INTO ACCOUNT!
active_learning_data.extract_dataset_from_pool(40000)

train_loader = torch.utils.data.DataLoader(
    active_learning_data.training_dataset,
    sampler=active_learning.RandomFixedLengthSampler(
        active_learning_data.training_dataset, training_iterations),
    batch_size=batch_size,
    **kwargs,
)

pool_loader = torch.utils.data.DataLoader(active_learning_data.pool_dataset,
                                          batch_size=scoring_batch_size,
                                          shuffle=False,
                                          **kwargs)

# Run experiment
test_accs = []
test_loss = []
added_indices = []

pbar = tqdm(initial=len(active_learning_data.training_dataset),
            total=max_training_samples,
            desc="Training Set Size")

while True:
    model = BayesianCNN(num_classes).to(device=device)
    optimizer = torch.optim.Adam(model.parameters())

    model.train()

    # Train
    for data, target in tqdm(train_loader, desc="Training", leave=False):
        data = data.to(device=device)
        target = target.to(device=device)

        optimizer.zero_grad()

        prediction = model(data, 1).squeeze(1)
        loss = F.nll_loss(prediction, target)

        loss.backward()
        optimizer.step()

    # Test
    loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader, desc="Testing", leave=False):
            data = data.to(device=device)
            target = target.to(device=device)

            prediction = torch.logsumexp(
                model(data, num_test_inference_samples),
                dim=1) - math.log(num_test_inference_samples)
            loss += F.nll_loss(prediction, target, reduction="sum")

            prediction = prediction.max(1)[1]
            correct += prediction.eq(target.view_as(prediction)).sum().item()

    loss /= len(test_loader.dataset)
    test_loss.append(loss)

    percentage_correct = 100.0 * correct / len(test_loader.dataset)
    test_accs.append(percentage_correct)

    print("Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)".format(
        loss, correct, len(test_loader.dataset), percentage_correct))

    if len(active_learning_data.training_dataset) >= max_training_samples:
        break

    # Acquire pool predictions
    N = len(active_learning_data.pool_dataset)
    logits_N_K_C = torch.empty((N, num_inference_samples, num_classes),
                               dtype=torch.double,
                               pin_memory=use_cuda)

    with torch.no_grad():
        model.eval()

        for i, (data, _) in enumerate(
                tqdm(pool_loader,
                     desc="Evaluating Acquisition Set",
                     leave=False)):
            data = data.to(device=device)

            lower = i * pool_loader.batch_size
            upper = min(lower + pool_loader.batch_size, N)
            logits_N_K_C[lower:upper].copy_(model(
                data, num_inference_samples).double(),
                                            non_blocking=True)

    with torch.no_grad():
        candidate_batch = batchbald.get_batchbald_batch(logits_N_K_C,
                                                        acquisition_batch_size,
                                                        num_samples,
                                                        dtype=torch.double,
                                                        device=device)

    targets = repeated_mnist.get_targets(active_learning_data.pool_dataset)
    dataset_indices = active_learning_data.get_dataset_indices(
        candidate_batch.indices)

    print("Dataset indices: ", dataset_indices)
    print("Scores: ", candidate_batch.scores)
    print("Labels: ", targets[candidate_batch.indices])

    active_learning_data.acquire(candidate_batch.indices)
    added_indices.append(dataset_indices)
    pbar.update(len(dataset_indices))

use_cuda: True


Test set: Average loss: 1.8308, Accuracy: 6338/10000 (63.38%)


Dataset indices:  [ 8289  3582 53863 25823  8257]
Scores:  [1.3557736757796552, 2.519825665002863, 3.4085050062563003, 3.9695525038521966, 4.2919329737575325]
Labels:  tensor([0, 2, 3, 0, 2])


Test set: Average loss: 1.4175, Accuracy: 6985/10000 (69.85%)


Dataset indices:  [52012 41383 13682 42198  6185]
Scores:  [1.3061743472090992, 2.3644483581086218, 3.221506615442796, 3.8194846029077567, 4.183660860241003]
Labels:  tensor([8, 0, 8, 4, 3])


...

Test set: Average loss: 0.3089, Accuracy: 9146/10000 (91.46%)


Dataset indices:  [34946 34520 49523 53873 33812]
Scores:  [1.0336427348167723, 1.9970519915069573, 2.797597657927694, 3.42035327890408, 3.8533839231792015]
Labels:  tensor([8, 6, 7, 4, 6])


Test set: Average loss: 0.3097, Accuracy: 9186/10000 (91.86%)


Dataset indices:  [44898  2761 57882 50317 41453]
Scores:  [0.9905177855898044, 1.9297613872862898, 2.7047249345431563, 3.3479105773782156, 3.7841506750832803]
Labels:  tensor([2, 8, 0, 3, 3])


Test set: Average loss: 0.2967, Accuracy: 9241/10000 (92.41%)


In [ ]:
# hide
test_accs

[63.38,
 69.85,
 72.69,
 74.35,
 78.09,
 80.67,
 82.67,
 83.97,
 83.86,
 85.82,
 83.67,
 86.21,
 84.66,
 87.54,
 87.63,
 88.03,
 88.89,
 90.09,
 89.94,
 91.24,
 92.34,
 92.64,
 92.82,
 92.61,
 91.46,
 91.86,
 92.41]

In [ ]:
# hide
test_loss

[tensor(1.8308, device='cuda:0'),
 tensor(1.4175, device='cuda:0'),
 tensor(1.2743, device='cuda:0'),
 tensor(1.1519, device='cuda:0'),
 tensor(0.9635, device='cuda:0'),
 tensor(0.7743, device='cuda:0'),
 tensor(0.6680, device='cuda:0'),
 tensor(0.6514, device='cuda:0'),
 tensor(0.5701, device='cuda:0'),
 tensor(0.5139, device='cuda:0'),
 tensor(0.5993, device='cuda:0'),
 tensor(0.5477, device='cuda:0'),
 tensor(0.6217, device='cuda:0'),
 tensor(0.4452, device='cuda:0'),
 tensor(0.4564, device='cuda:0'),
 tensor(0.4157, device='cuda:0'),
 tensor(0.3696, device='cuda:0'),
 tensor(0.3524, device='cuda:0'),
 tensor(0.3498, device='cuda:0'),
 tensor(0.3283, device='cuda:0'),
 tensor(0.2814, device='cuda:0'),
 tensor(0.2732, device='cuda:0'),
 tensor(0.2742, device='cuda:0'),
 tensor(0.2854, device='cuda:0'),
 tensor(0.3089, device='cuda:0'),
 tensor(0.3097, device='cuda:0'),
 tensor(0.2967, device='cuda:0')]

In [ ]:
# hide
added_indices

[array([ 8289,  3582, 53863, 25823,  8257]),
 array([52012, 41383, 13682, 42198,  6185]),
 array([11657, 37137, 14866, 28222, 34614]),
 array([39411, 13642, 19396,  8488, 16077]),
 array([40057,  4606, 55743, 26444, 37870]),
 array([ 2748, 25910, 24223, 32954, 20110]),
 array([56615, 37249, 50461,   384, 32509]),
 array([47695, 30925, 50010,  3916, 45114]),
 array([38760, 54950,  6578, 52959, 44590]),
 array([12281, 20820, 20859, 29132, 52972]),
 array([49202, 59080, 19590, 14699, 11295]),
 array([17756, 42467, 49580, 34902, 14705]),
 array([28860, 17010,  5217, 47426, 25117]),
 array([59674, 40066, 29303, 30878, 43852]),
 array([59314, 53260, 52697, 27356,  8668]),
 array([24440, 59390, 18412, 32178,  5852]),
 array([31184, 47652,  5129, 28731, 51261]),
 array([39151, 25387, 31637, 56391, 22675]),
 array([40766, 35638,  7768, 21390, 13021]),
 array([44202, 26034,  2765, 36818, 29489]),
 array([37293, 18398, 33126, 43126, 30454]),
 array([23086, 37414,  3070,  8214,  3218]),
 array([35

In [ ]:
# hide
# experiment
!pip install matplotlib

     |████████████████████████████████| 12.4 MB 6.6 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 10.6 MB/s eta 0:00:01
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)


In [ ]:
# experiment 
import matplotlib.pyplot as plt
import numpy as np

plt.plot(np.arange(num_initial_samples, max_training_samples + 1, acquisition_batch_size), test_accs)
plt.xlabel("# training samples")
plt.ylabel("Accuracy")
plt.vlines(105, 0, 100, linestyles="dashed", color="r")
plt.show()

NameError: name 'num_initial_samples' is not defined